In [ ]:
#export
from local.imports import *
from local.test import *
from local.core import *
from local.data.all import *
from local.notebook.showdoc import show_doc

In [ ]:
#export
from pandas.api.types import is_numeric_dtype

In [ ]:
#default_exp tabular.core

# Tabular core

> Basic function to preprocess tabular data before assembling it in a `DataBunch`.

## TabularProc -

In [ ]:
#export
@docs
class TabularProc():
    "Base class to write a tabular processor for dataframes"
    setup,__call__,order = noops,noops,0
    def __init__(self, func=None): self.__call__ = ifnone(func, self.__call__)
    
    _docs = dict(setup="Use `df` to set its state, with just `trn_idx` if passed",
                 __call__="Process `df` with the state computed during setup`")

We use this class to preprocess tabular data. `cat_names` should contain the names of the categorical variables in your dataframe, `cont_names` the names of the continuous variables. If you don't need any state, you can initiliaze a `TabularProc` with a `func` to be applied on the dataframes. Otherwise you should subclass and implement `setup` and `__call__`.

In [ ]:
#export
class Tabular(CollBase):
    def __init__(self, df, cat_names=None, cont_names=None, cat_y=None):
        super().__init__(df)
        self.cat_names,self.cont_names,self.cat_y = L(cat_names),L(cont_names),cat_y
    
    def __setitem__(self,k,v): super().__setitem__(list(k) if isinstance(k,L) else k, v)
    def transform(self, cols, f): self[cols] = self[cols].transform(f)
            
    @property
    def loc(self): return self.items.loc
    @property
    def iloc(self): return self.items.iloc
    
    @property
    def all_cat_names(self): return self.cat_names + self.cat_y
    @property
    def all_col_names(self): return self.cont_names + self.all_cat_names
    @property
    def classes(self): return {n:'#na#'+L(c.cat.categories) for n,c in self.all_cats.items()}

In [ ]:
#export
def _add_prop(cls, nm):
    prop = property(lambda o: o.items[list(getattr(o,nm+'_names'))])
    setattr(cls, nm+'s', prop)
    def _f(o,v): o.items[list(getattr(o,nm+'_names'))] = v
    setattr(cls, nm+'s', prop.setter(_f))

_add_prop(Tabular, 'cat')
_add_prop(Tabular, 'all_cat')
_add_prop(Tabular, 'cont')
_add_prop(Tabular, 'all_col')

In [ ]:
#export
class Categorify(TabularProc):
    "Transform the categorical variables to that type."
    order = 1
    def setup(self, to, trn_idx=None):
        self.categories = {}
        for n in to.all_cat_names:
            col = to.loc[ifnone(trn_idx, slice(None)),n]
            self.categories[n] = pd.Categorical(col, ordered=True).categories
        
    def __call__(self, to):
        to.transform(to.all_cat_names, lambda c: pd.Categorical(c, categories=self.categories[c.name], ordered=True))

In [ ]:
show_doc(Categorify, title_level=3)

<h3 id="<code>class</code> <code>Categorify</code>" class="doc_header"><code>class</code> <code>Categorify</code><a href="https://nbviewer.jupyter.org/github/fastai/fastai_docs/blob/master/dev/40_tabular_core.ipynb#TabularProc--" class="source_link" style="float:right">[source]</a></h3>

> <code>Categorify</code>(**`func`**=*`None`*) :: [`TabularProc`](/tabular.core.html#TabularProc)

Transform the categorical variables to that type.

In [ ]:
cat = Categorify()
df = pd.DataFrame({'a':[0,1,2,0,2]})
to = Tabular(df, 'a')

cat.setup(to)
test_eq(list(cat.categories['a']), [0,1,2])
cat(to)
test_eq(df['a'].cat.codes.values, np.array([0,1,2,0,2]))
df1 = pd.DataFrame({'a':[1,0,3,-1,2]})
to1 = Tabular(df1, 'a')
cat(to1)
#Values that weren't in the training df are sent to -1 (na)
test_eq(df1['a'].cat.codes.values, np.array([1,0,-1,-1,2]))

In [ ]:
cat = Categorify()
df = pd.DataFrame({'a':[0,1,2,3,2]})
to = Tabular(df, 'a')
cat.setup(to, trn_idx=[0,1,2])
test_eq(list(cat.categories['a']), [0,1,2])
cat(to)
test_eq(df['a'].cat.codes.values, np.array([0,1,2,-1,2]))

In [ ]:
#export
class Normalize(TabularProc):
    "Normalize the continuous variables."
    order = 2
    def setup(self, to, trn_idx=None):
        df = to.loc[ifnone(trn_idx,slice(None)), to.cont_names]
        self.means,self.stds = df.mean(),df.std(ddof=0)
    
    def __call__(self, to): to.conts = (to.conts-self.means) / (self.stds+1e-7)

In [ ]:
show_doc(Normalize, title_level=3)

<h3 id="<code>class</code> <code>Normalize</code>" class="doc_header"><code>class</code> <code>Normalize</code><a href="https://nbviewer.jupyter.org/github/fastai/fastai_docs/blob/master/dev/40_tabular_core.ipynb#TabularProc--" class="source_link" style="float:right">[source]</a></h3>

> <code>Normalize</code>(**`func`**=*`None`*) :: [`TabularProc`](/tabular.core.html#TabularProc)

Normalize the continuous variables.

In [ ]:
norm = Normalize()
df = pd.DataFrame({'a':[0,1,2,3,4]})
to = Tabular(df, cont_names='a')
norm.setup(to)
x = np.array([0,1,2,3,4])
m,s = x.mean(),x.std()
test_eq(norm.means['a'], m)
test_close(norm.stds['a'], s)
norm(to)
test_close(df['a'].values, (x-m)/s)
df1 = pd.DataFrame({'a':[5,6,7]})
to1 = Tabular(df1, cont_names='a')
norm(to1)
test_close(df1['a'].values, (np.array([5,6,7])-m)/s)

In [ ]:
norm = Normalize()
df = pd.DataFrame({'a':[0,1,2,3,4]})
to = Tabular(df, cont_names='a')
norm.setup(to, trn_idx=[0,1,2])
x = np.array([0,1,2])
m,s = x.mean(),x.std()
test_eq(norm.means['a'], m)
test_close(norm.stds['a'], s)
norm(to)
test_close(df['a'].values, (np.array([0,1,2,3,4])-m)/s)

In [ ]:
#export
mk_class('FillStrategy', **{o:o for o in ['median', 'constant', 'most_common']})
FillStrategy.__doc__ = "Namespace containing the various filling strategies"

In [ ]:
#export
_all_ = ['FillStrategy']

In [ ]:
show_doc(FillStrategy, title_level=3)

<h3 id="<code>class</code> <code>FillStrategy</code>" class="doc_header"><code>class</code> <code>FillStrategy</code><a href="" class="source_link" style="float:right">[source]</a></h3>

> <code>FillStrategy</code>(**\*`args`**, **\*\*`kwargs`**)

Namespace containing the various filling strategies

In [ ]:
#export
class FillMissing(TabularProc):
    "Fill the missing values in continuous columns."
    def __init__(self, fill_strategy=FillStrategy.median, add_col=True, fill_val=0.):
        store_attr(self, 'fill_strategy,add_col,fill_val')
    
    def setup(self, to, trn_idx=None):
        self.na_dict = {}
        df = to.loc[ifnone(trn_idx,slice(None)), to.cont_names]
        for n in to.cont_names:
            col = df[n]
            if pd.isnull(col).sum():
                self.na_dict[n] = {
                    FillStrategy.median:      col.median(),
                    FillStrategy.constant:    self.fill_val,
                    FillStrategy.most_common: col.dropna().value_counts().idxmax()
                }[self.fill_strategy]

    def __call__(self, to):
        for n in to.cont_names:
            if n in self.na_dict:
                if self.add_col: 
                    to[n+'_na'] = pd.isnull(to[n])
                    if n+'_na' not in to.cat_names: to.cat_names.append(n+'_na')
                to[n] = to[n].fillna(self.na_dict[n])
            elif pd.isnull(to[n]).sum() != 0:
                raise Exception(f"""There are nan values in field {n} but there were none in the training set given at setup. 
                Please fix those manually.""")

In [ ]:
show_doc(FillMissing, title_level=3)

<h3 id="<code>class</code> <code>FillMissing</code>" class="doc_header"><code>class</code> <code>FillMissing</code><a href="https://nbviewer.jupyter.org/github/fastai/fastai_docs/blob/master/dev/40_tabular_core.ipynb#TabularProc--" class="source_link" style="float:right">[source]</a></h3>

> <code>FillMissing</code>(**`fill_strategy`**=*`'median'`*, **`add_col`**=*`True`*, **`fill_val`**=*`0.0`*) :: [`TabularProc`](/tabular.core.html#TabularProc)

Fill the missing values in continuous columns.

In [ ]:
fill1,fill2,fill3 = (FillMissing(fill_strategy=s) 
                     for s in [FillStrategy.median, FillStrategy.constant, FillStrategy.most_common])
df = pd.DataFrame({'a':[0,1,np.nan,1,2,3,4]})
df1 = df.copy(); df2 = df.copy()
to,to1,to2 = Tabular(df, cont_names='a'),Tabular(df1, cont_names='a'),Tabular(df2, cont_names='a')
fill1.setup(to); fill2.setup(to1); fill3.setup(to2)
test_eq(fill1.na_dict, {'a': 1.5})
test_eq(fill2.na_dict, {'a': 0})
test_eq(fill3.na_dict, {'a': 1.0})

fill1(to); fill2(to1); fill3(to2)
for t in [to, to1, to2]: test_eq(t.cat_names, ['a_na'])

for df_,v in zip([to, to1, to2], [1.5, 0., 1.]):
    test_eq(df_['a'].values, np.array([0, 1, v, 1, 2, 3, 4]))
    test_eq(df_['a_na'].values, np.array([0, 0, 1, 0, 0, 0, 0]))
    
dfa = pd.DataFrame({'a':[np.nan,0,np.nan]})
dfa1 = dfa.copy(); dfa2 = dfa.copy()
to,to1,to2 = Tabular(dfa, cont_names='a'),Tabular(dfa1, cont_names='a'),Tabular(dfa2, cont_names='a')
fill1(to); fill2(to1); fill3(to2)
for df_,v in zip([to, to1, to2], [1.5, 0., 1.]):
    test_eq(df_['a'].values, np.array([v, 0, v]))
    test_eq(df_['a_na'].values, np.array([1, 0, 1]))

## TabularProcessor -

In [ ]:
#export
class TabularPreprocessor(GetAttr):
    "An object that will preprocess dataframes using `procs`"
    def __init__(self, procs, inplace=True):
        self.inplace = inplace
        procs = L(procs).sorted(key='order')
        self.procs = {p:(p() if isinstance(p, type) else TabularProc(func=p)) for p in procs}
    
    def __call__(self, to, trn_idx=None):
        "Call each of `self.procs` on `df`, setup on `df[trn_idx]` if not None"
        if not self.inplace: to.items = to.items.copy()
        if trn_idx is None:
            for p in self.procs.values(): p(to)
        else:
            self.default = self.to = to
            self.procs,procs = {},self.procs
            for t,p in procs.items():
                p.setup(to, trn_idx=trn_idx)
                p(to)
                self.procs[t] = p

In [ ]:
procs = [Normalize, Categorify, FillMissing, noop]
proc = TabularPreprocessor(procs, inplace=False)

#Test reordering and partialize
test_eq(proc.procs.keys(), [FillMissing, noop, Categorify, Normalize])

df = pd.DataFrame({'a':[0,1,2,1,1,2,0], 'b':[0,1,np.nan,1,2,3,4]})
to = Tabular(df, 'a', 'b')

#Test setup and apply on df_trn
proc(to, trn_idx=range_of(df))
test_eq(to['a'].cat.codes, [0,1,2,1,1,2,0])
test_eq(to['b_na'].cat.codes, [0,0,1,0,0,0,0])
x = np.array([0,1,1.5,1,2,3,4])
m,s = x.mean(),x.std()
test_close(to['b'].values, (x-m)/s)
test_eq(proc.classes, {'a': ['#na#',0,1,2], 'b_na': ['#na#',False,True]})

In [ ]:
#Test apply on df_val
df = pd.DataFrame({'a':[2,1,3], 'b':[4,5,np.nan]})
to = Tabular(df, 'a', 'b')
proc(to)
test_eq(proc.classes, {'a': ['#na#',0,1,2], 'b_na': ['#na#',False,True]})
test_eq(to['a'].cat.codes, [2,1,-1])
test_eq(to['b_na'].cat.codes, [0,0,1])
x = np.array([4, 5, 1.5])
test_close(to['b'].values, (x-m)/s)

In [ ]:
#Test apply on cat_y
procs = [Normalize, Categorify, FillMissing, noop]
proc = TabularPreprocessor(procs, inplace=False)

df = pd.DataFrame({'a':[0,1,2,1,1,2,0], 'b':[0,1,np.nan,1,2,3,4], 'c': ['b','a','b','a','a','b','a']})
to = Tabular(df, 'a', 'b', cat_y='c')
proc(to, trn_idx=range_of(df))
test_eq(to.cat_names, ['a', 'b_na'])
test_eq(to['a'].cat.codes, [0,1,2,1,1,2,0])
test_eq(to['b_na'].cat.codes, [0,0,1,0,0,0,0])
test_eq(to['c'].cat.codes, [1,0,1,0,0,1,0])
x = np.array([0,1,1.5,1,2,3,4])
m,s = x.mean(),x.std()
test_close(to['b'].values, (x-m)/s)
test_eq(proc.classes, {'a': ['#na#',0,1,2], 'b_na': ['#na#',False,True], 'c': ['#na#','a','b']})

In [ ]:
#export
def process_df(df, splits, procs, cat_names=None, cont_names=None, cat_y=None, inplace=True):
    "Process `df` with `procs` and returns the processed dataframe and the `TabularProcessor` associated"
    to = Tabular(df, cat_names, cont_names, cat_y)
    proc = TabularPreprocessor(procs, inplace=inplace)
    proc(to, trn_idx=splits[0])
    return to,proc

In [ ]:
procs = [Normalize, Categorify, FillMissing, noop]
df = pd.DataFrame({'a':[0,1,2,1,1,2,0], 'b':[0,1,np.nan,1,2,3,4], 'c': ['b','a','b','a','a','b','a']})
to,proc = process_df(df, [range_of(df)], procs=procs, cat_names='a', cont_names='b', cat_y='c')
test_eq(to.cat_names, ['a', 'b_na'])
test_eq(to['a'].cat.codes, [0,1,2,1,1,2,0])
test_eq(to['b_na'].cat.codes, [0,0,1,0,0,0,0])
test_eq(to['c'].cat.codes, [1,0,1,0,0,1,0])

Pass the same `splits` as you will use for splitting the data, so that the setup is only done on the training set. `cat_names` are the names of the categorical variables, `cont_names` the continous ones, `cat_y` are the names of the dependent variables that are categories. If `inplace=True`, processing is applied inplace, otherwis it creates a copy of `df`.

In [ ]:
#export
class TabularLine(pd.Series):
    "A line of a dataframe that knows how to show itself"
    def show(self, ctx=None, **kwargs):
        if ctx is None: return self
        else: return ctx.append(self)

In [ ]:
#export
class TensorTabular(tuple):
    
    def get_ctxs(self, max_samples=10, **kwargs):
        n_samples = min(self[0].shape[0], max_samples)
        df = pd.DataFrame(index = range(n_samples))
        return [df.iloc[i] for i in range(n_samples)]
    
    def display(self, ctxs): display_df(pd.DataFrame(ctxs))

In [ ]:
#export
class ReadTabLine(ItemTransform):
    def __init__(self, proc): 
        self.proc = proc
        self.o2is = {n: defaultdict(int, {v:i for i,v in enumerate(proc.classes[n])}) for n in proc.cat_names}
    
    def encodes(self, row): 
        cats = [self.o2is[n][row[n]] for n in self.proc.cat_names]
        conts = [row[n] for n in self.proc.cont_names]
        return TensorTabular((tensor(cats).long(),tensor(conts).float()))
    
    def decodes(self, o) -> TabularLine:
        dic = {c: self.proc.classes[c][v] for v,c in zip(o[0], self.proc.cat_names)}
        norm = self.proc.procs.get(Normalize, None)
        if norm is not None:
            dic.update({c: (v*norm.stds[c] + norm.means[c]).item() for v,c in zip(o[1], self.proc.cont_names)})
        return pd.Series(dic)

In [ ]:
#export
class ReadTabTarget(ItemTransform):
    def __init__(self, proc): 
        self.proc = proc
        self.o2i = defaultdict(int, {v:i for i,v in enumerate(proc.classes[proc.cat_y])})
    
    def encodes(self, row): return self.o2i[row[self.proc.cat_y]]-1
    def decodes(self, o) -> Category: return self.proc.classes[self.proc.cat_y][o+1]

In [ ]:
tds = TfmdDS(to.items, tfms=[[ReadTabLine(proc)], ReadTabTarget(proc)], use_list=None)

In [ ]:
enc = tds[1]
test_eq(enc[0][0], tensor([2,1]))
test_close(enc[0][1], tensor([-0.628828]))
test_eq(enc[1], 0)

dec = tds.decode(enc)
assert isinstance(dec[0], TabularLine)
test_eq(dec[0], pd.Series({'a': 1, 'b_na': False, 'b': 1}))
test_eq(dec[1], 'a')

test_stdout(lambda: print(tds.show_at(1)), """a               1
b_na        False
b               1
category        a
dtype: object""")

## Integration example

In [ ]:
path = untar_data(URLs.ADULT_SAMPLE)
df = pd.read_csv(path/'adult.csv')
df_trn,df_tst = df.iloc[:10000],df.iloc[10000:]
df_trn.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,salary
0,49,Private,101320,Assoc-acdm,12.0,Married-civ-spouse,NaN,Wife,White,Female,0,1902,40,United-States,>=50k
1,44,Private,236746,Masters,14.0,Divorced,Exec-managerial,Not-in-family,White,Male,10520,0,45,United-States,>=50k
2,38,Private,96185,HS-grad,NaN,Divorced,NaN,Unmarried,Black,Female,0,0,32,United-States,<50k
3,38,Self-emp-inc,112847,Prof-school,15.0,Married-civ-spouse,Prof-specialty,Husband,Asian-Pac-Islander,Male,0,0,40,United-States,>=50k
4,42,Self-emp-not-inc,82297,7th-8th,NaN,Married-civ-spouse,Other-service,Wife,Black,Female,0,0,50,United-States,<50k


In [ ]:
cat_names = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race']
cont_names = ['age', 'fnlwgt', 'education-num']
procs = [Categorify, FillMissing, Normalize]

In [ ]:
splits = RandomSplitter()(range_of(df_trn))
to,proc = process_df(df_trn, splits, procs=procs, cat_names=cat_names, cont_names=cont_names, cat_y="salary", inplace=False)

In [ ]:
dsrc = DataSource(to.items, filts=splits, tfms=[[ReadTabLine(proc)], [ReadTabTarget(proc)]])

In [ ]:
dbch = dsrc.databunch(bs=64)
dbch.show_batch()

,workclass,education,marital-status,occupation,relationship,race,education-num_na,age,fnlwgt,education-num,category
0,Private,HS-grad,Divorced,Sales,Not-in-family,White,False,42.0,97688.007812,9.0,<50k
1,Private,7th-8th,Never-married,Handlers-cleaners,Not-in-family,White,False,58.0,192806.000000,4.0,<50k
2,State-gov,Bachelors,Married-civ-spouse,Protective-serv,Husband,White,False,32.0,182556.000000,13.0,>=50k
3,Local-gov,7th-8th,Married-civ-spouse,Transport-moving,Husband,White,False,51.0,97005.007812,4.0,<50k
4,Private,Bachelors,Divorced,Prof-specialty,Not-in-family,White,False,40.0,105936.007812,13.0,<50k
5,Private,HS-grad,Never-married,Machine-op-inspct,Own-child,White,False,20.0,131230.000000,9.0,<50k
6,Private,Some-college,Married-civ-spouse,Other-service,Husband,White,False,31.0,59083.000000,10.0,<50k
7,Local-gov,Some-college,Divorced,Adm-clerical,Unmarried,White,False,36.0,331902.000000,10.0,<50k
8,Private,Some-college,Never-married,Other-service,Not-in-family,White,False,33.0,256362.000000,10.0,<50k
9,Private,HS-grad,Married-civ-spouse,Other-service,Husband,White,False,25.0,315643.000000,9.0,<50k


In [ ]:
to_tst = Tabular(df_tst, cat_names, cont_names, cat_y="salary")
proc(to_tst)
to_tst.all_cols.head()

,age,fnlwgt,education-num,workclass,education,marital-status,occupation,relationship,race,education-num_na,salary
10000,0.455639,1.339722,1.157686,Private,Bachelors,Married-civ-spouse,Adm-clerical,Husband,Asian-Pac-Islander,False,<50k
10001,-0.937079,1.252712,-0.433498,Private,HS-grad,Married-civ-spouse,Transport-moving,Husband,Other,False,<50k
10002,1.042046,0.150204,-1.229090,Private,11th,Divorced,Other-service,Not-in-family,White,False,<50k
10003,0.528940,-0.282833,-0.433498,Private,HS-grad,Widowed,Adm-clerical,Unmarried,White,False,<50k
10004,0.748842,1.442240,0.362094,Self-emp-inc,Assoc-voc,Married-civ-spouse,Exec-managerial,Husband,White,False,>=50k


In [ ]:
to_tst.all_cols.dtypes

age                  float64
fnlwgt               float64
education-num        float64
workclass           category
education           category
marital-status      category
occupation          category
relationship        category
race                category
education-num_na    category
salary              category
dtype: object

## Export -

In [ ]:
#hide
from local.notebook.export import notebook2script
notebook2script(all_fs=True)

Converted 00_test.ipynb.
Converted 01_core.ipynb.
Converted 01a_dataloader.ipynb.
Converted 01a_script.ipynb.
Converted 02_transforms.ipynb.
Converted 03_pipeline.ipynb.
Converted 04_data_external.ipynb.
Converted 05_data_core.ipynb.
Converted 06_data_source.ipynb.
Converted 07_vision_core.ipynb.
Converted 08_pets_tutorial.ipynb.
Converted 09_vision_augment.ipynb.
Converted 11_layers.ipynb.
Converted 12_optimizer.ipynb.
Converted 13_learner.ipynb.
Converted 14_callback_schedule.ipynb.
Converted 15_callback_hook.ipynb.
Converted 16_callback_progress.ipynb.
Converted 17_callback_tracker.ipynb.
Converted 18_callback_fp16.ipynb.
Converted 19_callback_mixup.ipynb.
Converted 20_metrics.ipynb.
Converted 21_tutorial_imagenette.ipynb.
Converted 30_text_core.ipynb.
Converted 31_text_data.ipynb.
Converted 32_text_models_awdlstm.ipynb.
Converted 33_test_models_core.ipynb.
Converted 34_callback_rnn.ipynb.
Converted 35_tutorial_wikitext.ipynb.
Converted 36_text_models_qrnn.ipynb.
Converted 40_tabula